In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

In [2]:
a = Asset(c.BTC, c.USDT)

In [8]:
a.__dict__()

TypeError: 'dict' object is not callable

In [6]:
json.dumps(a.__dict__)

'{"base": "BTC", "quote": "USDT"}'

### CCXT

In [4]:
polo = ccxt.poloniex({
    'apiKey': cfg.POLONIEX_API_KEY,
    'secret': cfg.POLONIEX_API_SECRET_KEY,
})
gdax = ccxt.gdax({
    'apiKey': cfg.GDAX_API_KEY,
    'secret': cfg.GDAX_API_SECRET_KEY,
    'password': cfg.GDAX_PASSPHRASE,
    'verbose':False,
})
binance = ccxt.binance()

exchange = binance
markets = exchange.load_markets()

In [7]:
markets['BTC/USDT']

{'active': True,
 'base': 'BTC',
 'id': 'BTCUSDT',
 'info': {'baseAsset': 'BTC',
  'baseAssetPrecision': 8,
  'filters': [{'filterType': 'PRICE_FILTER',
    'maxPrice': '10000000.00000000',
    'minPrice': '0.01000000',
    'tickSize': '0.01000000'},
   {'filterType': 'LOT_SIZE',
    'maxQty': '10000000.00000000',
    'minQty': '0.00000100',
    'stepSize': '0.00000100'},
   {'filterType': 'MIN_NOTIONAL', 'minNotional': '1.00000000'}],
  'icebergAllowed': True,
  'orderTypes': ['LIMIT',
   'LIMIT_MAKER',
   'MARKET',
   'STOP_LOSS_LIMIT',
   'TAKE_PROFIT_LIMIT'],
  'quoteAsset': 'USDT',
  'quotePrecision': 8,
  'status': 'TRADING',
  'symbol': 'BTCUSDT'},
 'limits': {'amount': {'max': 10000000.0, 'min': 1e-06},
  'cost': {'max': None, 'min': 1.0},
  'price': {'max': 10000000.0, 'min': 0.01}},
 'lot': 1e-06,
 'maker': 0.001,
 'percentage': True,
 'precision': {'amount': 6, 'base': 8, 'price': 2, 'quote': 8},
 'quote': 'USDT',
 'symbol': 'BTC/USDT',
 'taker': 0.001,
 'tierBased': False}

### Ingest Data

In [ ]:
coins = [c.BTC, c.ETH, c.LTC]#, c.ETH, c.XRP, c.XMR, c.DASH]
market = c.USDT
coin = c.LTC
symbol = ohlcv.get_symbol(coin, market)
period = '1m' #1800
#start = datetime.datetime(year=2018, month=1, day=4, hour=14)
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, coin, market, period, start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(coin, market, exchange.id, period)
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, coins, market, period, start, end)
df = ohlcv.load_multi_coin_data(exchange.id, coins, market, period, start)
df.head()

### Feeds

In [ ]:
# CSV Feed
feed_fpath = ohlcv.get_price_data_fpath(coin=c.LTC, market=c.USDT, exchange_id=exchange.id, period='1m')
feed = CSVDataFeed(feed_fpath)
feed.initialize()

In [ ]:
# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])

In [ ]:
# Access all rows in history
feed.history().head()

In [ ]:
# Exchange Live Feed

coins = [c.ETH, c.XRP, c.XMR]
market = c.BTC
period = '1m'
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)
end = None
feed_fpath = ohlcv.get_price_data_fpath(coins[2], market, exchange.id, period)

In [ ]:
feed = ExchangeDataFeed(exchange, [coins[2]], market, period, feed_fpath, start, end)
feed.initialize()

In [ ]:
# Access History
feed.history(t_minus=3).tail()

In [ ]:
# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

In [ ]:
# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])

In [ ]:
# Example Using the Feed

def my_strategy(row, history, exchange):
    print("Time:", row['time_utc'], "Price:", row['close'])
    if row['close'] > .024:
        print("Buying coin")
        #res = exchange.limit_buy(quantity=1, price=5)
    else:
        print("Selling that stupid coin")
    return {
        'time': row['time_utc'],
        'close': row['close'],
        'action': 'buy',
        'quantity': 1,
        'price': 5,
        'exchange': exchange.id
    }

def backtest(feed, strategy, exchange):
    results = []
    row = feed.next()
    while row is not None:
        output = strategy(row, feed, exchange)
        row = feed.next(refresh=True)
        results.append(output)
    return results

In [ ]:
res = backtest(feed, my_strategy, exchange)

### Order

In [ ]:
order = Order(
    ex_id=exchange.id, 
    coin=c.LTC,
    market=c.USDT,
    price=250., 
    quantity=1, order_type=OrderType.BUY_LIMIT
)
order.set_status(OrderStatus.OPEN)
order

In [ ]:
order_json = order.to_json()
load_order_from_json(order_json)

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics

class Strategy():
    def __init__(self, context):
        self.ctx = context
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        pass
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

### Plotting

In [ ]:
utils.charts.plot_range(feed.history(), start, end, 'close')

### Exchange

In [ ]:
# https://github.com/ccxt/ccxt/blob/master/python/ccxt/base/exchange.py

class Exchange():
    def __init__(self):
        pass

class CCXTExchange(Exchange):
    def __init__(self, exchange_client):
        self.ex = exchange_client
        
    def fetch_balance(self):
        return self.ex.fetch_balance()
    
    def limit_buy(self, symbol, quantity, price):
        return self.ex.create_limit_buy_order(symbol, quantity, price)
    

class TestExchange(Exchange):
    def __init__(self):
        self.orders = {'BTC/USD':[]}
        self.trades = {}
        self.fees = {'trading': .025, 'funding': .025}
        self.balance = {
            'BTC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
            'USD': {'free': 0.6, 'total': 0.6, 'used': 0.0},
            'free': {'BCH': 0.0, 'BTC': 0.6, 'ETH': 0.0, 'LTC': 0.0, 'USD': 486.3}
        }
    
    def fetch_balance(self):
        return self.balance
    
    def limit_buy(self, coin, market, quantity, price):
        symbol = ohlcv.get_symbol(coin, market)
        self.orders[symbol].append("NEW_ORDER")
        
        # We assume the order goes through
        self.balance[market]['free'] -= quantity * price
        self.balance[coin]['used'] += quantity * price
        
        # Account for fees here
        self.balance[market]['total'] -= self.fees['trading']
        return self.orders

In [ ]:
ex = TestExchange()
print(ex.fetch_balance())
orders = ex.limit_buy(c.BTC, c.USD, 1, 15000.0)
orders

In [ ]:
EX = CCXTExchange(gdax)

### Runner

In [ ]:
@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2


class Punisher():
    def __init__(self, exchange, feed, strategy, record):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def backtest(self):
        results = []
        row = self.feed.next()
        while row is not None:
            print("Timestep", row['time_utc'], "Price", row['close'])
            row = self.feed.next()
            output = self.strategy(row, self.exchange, self.feed)
            results.append(output)
        return results
    
    def simulate(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['test'].append(output)
            time.sleep(2)

    def live(self):
        while True:
            row = self.feed.next()
            if row is not None:
                output = self.strategy(row, self.exchange, self.feed)
                self.record['live'].append(output)
            time.sleep(2)
    
    def punish(self, mode=False):
        if mode == TradeMode.BACKTEST:
            print("Backtesting ...")
            self.backtest()
        elif mode == TradeMode.SIMULATE:
            print("Simulating orders ...")
            self.simulate()
        elif mode == TradeMode.LIVE:
            print("LIVE TRADING! CAREFUL!")
            self.live()